<h1>Organisatorisches: Projekt mit schriftlicher Ausarbeitung</h1>

4. Gruppe:
Thema: Diamond Price Prediction
Vortragstag: 8.1.2025
Mitglieder: Nahid Qazi, Mohaddese Haydari, Azad Akin, Sean Müller, Denis Meyendrisch

### 1. Analyseziel:

- Prognose des Diamantenpreises basierend auf den verfügbaren Merkmalen.  
- Untersuchung der wichtigsten Einflussfaktoren (z.B. Karat, Schliff, Farbe, Reinheit) auf den Preis.  
- Modellierung und Evaluierung der Vorhersagegenauigkeit von mindestens zwei verschiedenen Machine-Learning-Modellen.  
- Visualisierung von Beziehungen und Verteilungen der Merkmale und ihrer Korrelation zum Preis.  

### 2. Kurze Beschreibung der Daten und der Datenqualität

Der vorliegende Datensatz enthält Informationen zu verschiedenen Eigenschaften von Diamanten, die für die Vorhersage ihres Preises verwendet werden. Jedes Datensatzobjekt beschreibt einen einzelnen Diamanten anhand der folgenden Merkmale:

- **Carat (Karat)**: Das Gewicht des Diamanten, ein wichtiger Faktor für den Preis.
- **Cut (Schliff)**: Die Qualität des Schliffs des Diamanten, unterteilt in verschiedene Kategorien wie "Ideal", "Premium", "Good", "Very Good" und "Fair".
- **Color (Farbe)**: Die Farbqualität des Diamanten, bewertet mit einer Skala von D (bestes Weiß) bis Z (farbiger Diamant).
- **Clarity (Reinheit)**: Die Reinheit des Diamanten, die die Anzahl und Sichtbarkeit von Einschlüsse und Unregelmäßigkeiten beschreibt, mit verschiedenen Kategorien wie "VVS1", "VS2", "SI1", "SI2", "I1".
- **Depth (Tiefe)**: Die Tiefe des Diamanten in Prozent.
- **Table (Tabelle)**: Der Anteil der oberen Fläche des Diamanten im Verhältnis zum Durchmesser.
- **Price (Preis)**: Der Marktpreis des Diamanten, der als Zielvariable für die Vorhersage dient.
- **x, y, z**: Die physikalischen Dimensionen des Diamanten (Länge, Breite, Höhe in Millimetern).

Die Daten sind von guter Qualität, da es **keine fehlenden Werte** gibt. Alle Werte sind vollständig und es gibt keine offensichtlichen Inkonsistenzen oder fehlerhaften Einträge. Allerdings sollten die numerischen Merkmale wie `carat`, `depth`, `table` und `price` einer weiteren Überprüfung auf Ausreißer unterzogen werden, um sicherzustellen, dass diese Werte in einem angemessenen Bereich liegen und keine Fehler vorliegen. Kategorische Variablen wie `cut`, `color` und `clarity` wurden bereits in einem standardisierten Format vorverarbeitet.

Insgesamt weist der Datensatz keine fehlenden oder ungültigen Daten auf, was eine gute Grundlage für die Durchführung von Modellierung und Vorhersagen bietet.

#### Import der erforderlichen Module und des Datensatzes

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, MinMaxScaler

# Get the directory where the current notebook is located
notebook_dir = os.path.dirname(os.path.abspath("__file__"))

# Define the relative path to the dataset
dataset_path = os.path.join(notebook_dir, 'Datasets', 'Diamonds.csv')

# Load the dataset
df = pd.read_csv(dataset_path, sep=',', index_col=0)

# Display the first few rows of the dataset
df.head()

### 3. Data Cleaning


#### 3.1 Data Cleaning

In [260]:
df['cut'] = df['cut'].str.title()
df['color'] = df['color'].str.upper()
df['clarity'] = df['clarity'].str.upper()

In [ ]:
# Boxplot für Ausreißer
df.boxplot(column=['price'], figsize=(6, 4))
plt.title("Ausreißerprüfung für Preis")
plt.show()

df.boxplot(column=['carat'], figsize=(6, 4))
plt.title("Ausreißerprüfung für Carat")
plt.show()


In [262]:
# Kopie von der Originaldatei erstellen, bevor man was drauf speichert/ändert
df_cleaned = df.copy()


In [ ]:
# IQR-Methode zur Entfernung von Ausreißern
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

# Grenzen definieren
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Entferne Ausreißer
df_cleaned = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]

# Zeilenanzahl vor und nach der Bereinigung
print("Anzahl der Zeilen vor der Bereinigung:", len(df))
print("Anzahl der Zeilen nach der Bereinigung:", len(df_cleaned))


In [ ]:
df.boxplot(column=['price'], by='cut', figsize=(6, 4))
plt.title("Bereinigter Preis pro 'cut'")
plt.show()


In [265]:
# Definiere den Pfad zum Ordner "Datasets"
datasets_folder = os.path.join(os.getcwd(), "Datasets")

# Speichern des bereinigten Datensatzes im Ordner "Datasets"
df_cleaned.to_csv(os.path.join(datasets_folder, "diamonds_cleaned.csv"), index=False)



#### 3.2 Missing Value Imputation

In [ ]:
df.isnull().sum()


-> keine Imputation nötig

### 4. EDA


#### 4.1 Feature Verteilungen, Korrelationen, Visualisierungen


In [ ]:
# Kategorische Werte analysieren
CategorialFeatures = ["cut", "color", "clarity"]

# Häufigkeiten analysieren
for feature in CategorialFeatures:
    print(f"Häufigkeit für {feature}:")
    value_counts = df[feature].value_counts()

    # Formatierte Ausgabe
    styled_counts = value_counts.to_frame(name="Anzahl").style.highlight_max(axis=0, color="lightgreen").highlight_min(axis=0, color="lightcoral")
    display(styled_counts)  # Anzeige in Jupyter-Notebook

    # Kategorische Werte visuell darstellen mit Barplots
    plt.figure(figsize=(6, 4))
    sns.countplot(data=df, x=feature, palette="coolwarm")
    plt.title(f"Häufigkeit der Kategorien in '{feature}'")
    plt.show()


In [ ]:
# Häufigkeiten für kategorische Werte
CategorialFeatures = ["cut", "color", "clarity"]
for feature in CategorialFeatures:
    print(f"Häufigkeit für {feature}:")
    value_counts = df[feature].value_counts()

    # Formatierte Ausgabe
    styled_counts = value_counts.to_frame(name="Anzahl").style.highlight_max(axis=0, color="lightgreen").highlight_min(axis=0, color="lightcoral")
    display(styled_counts)  # Anzeige in Jupyter-Notebook

    # Balkendiagramme für kategorische Werte
    df[feature].value_counts().plot(kind='bar', figsize=(5, 3), title=f"Häufigkeit von '{feature}'", color='skyblue', edgecolor='black')
    plt.xlabel(feature.capitalize())
    plt.ylabel("Anzahl")
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()



In [ ]:
# Liste der numerischen Features
QuantitativeFeatures = ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']

# Histogramm -> hier ist Rice-Regel von Vorteil für jedes numerische Feature
for feature in QuantitativeFeatures:
    df[feature].hist(bins="rice", figsize=(5, 3))
    plt.title(f"Verteilung von '{feature}' (Rice-Regel)")
    plt.xlabel(feature)
    plt.ylabel("Anzahl")
    plt.show()


In [ ]:
# Statistische Zusammenfassung für numerische Features
QuantitativeFeatures = ["carat", "depth", "table", "price", "x", "y", "z"]
summary_stats = df[QuantitativeFeatures].describe()

# Stilvolle Ausgabe
styled_summary = summary_stats.style.format(precision=2).highlight_max(axis=0, color="lightgreen").highlight_min(axis=0, color="lightcoral")
display(styled_summary)


In [ ]:
# Kategorische Features miteinander vergleichen
# Kreuztabellen für Kategorische Features
crosstab_cut_color = pd.crosstab(df['cut'], df['color'])
crosstab_cut_clarity = pd.crosstab(df['cut'], df['clarity'])
crosstab_color_clarity = pd.crosstab(df['color'], df['clarity'])

print("Kreuztabelle zwischen 'cut' und 'color':\n", crosstab_cut_color)
print("\nKreuztabelle zwischen 'cut' und 'clarity':\n", crosstab_cut_clarity)
print("\nKreuztabelle zwischen 'color' und 'clarity':\n", crosstab_color_clarity)


In [ ]:
# Gestapeltes Balkendiagramm für Cut und Color
crosstab_cut_color.plot(kind='bar', stacked=True, figsize=(8, 5))
plt.title("Korrelation zwischen 'cut' und 'color'")
plt.xlabel("Cut")
plt.ylabel("Anzahl")
plt.legend(title="Color")
plt.show()

# Gestapeltes Balkendiagramm für Cut und Clarity
crosstab_cut_clarity.plot(kind='bar', stacked=True, figsize=(8, 5))
plt.title("Korrelation zwischen 'cut' und 'clarity'")
plt.xlabel("Cut")
plt.ylabel("Anzahl")
plt.legend(title="Clarity")
plt.show()

# Gestapeltes Balkendiagramm für Color und Clarity
crosstab_color_clarity.plot(kind='bar', stacked=True, figsize=(8, 5))
plt.title("Korrelation zwischen 'color' und 'clarity'")
plt.xlabel("Color")
plt.ylabel("Anzahl")
plt.legend(title="Clarity")
plt.show()


In [273]:
# Definiere den Pfad zum Ordner "Datasets"
datasets_folder = os.path.join(os.getcwd(), "Datasets")

crosstab_cut_color.to_csv(os.path.join(datasets_folder, "cut_vs_color.csv"))
crosstab_cut_clarity.to_csv(os.path.join(datasets_folder, "cut_vs_clarity.csv"))
crosstab_color_clarity.to_csv(os.path.join(datasets_folder, "color_vs_clarity.csv"))

In [ ]:
# Quantitative Werte mit Quantitativen Werten vergleichen (Scatterplots)
QuantitativeFeatures = ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']

for i, feature_x in enumerate(QuantitativeFeatures):
    for j, feature_y in enumerate(QuantitativeFeatures):
        if i < j:  # Vermeidung von doppelten Kombinationen
            df.plot.scatter(x=feature_x, y=feature_y, figsize=(6, 4))
            plt.title(f"{feature_x} vs. {feature_y}")
            plt.show()


In [ ]:
# Korrelationstabelle für numerische Features -> Wie hängen numerische Features miteinander zusammen
# Werte zwischen -1 und +1: 
# +1: Starke positive Korrelation (beide Werte steigen gemeinsam).
# -1: Starke negative Korrelation (ein Wert steigt, der andere sinkt).
# 0: Keine lineare Korrelation.
# Korrelationstabelle für numerische Features
QuantitativeFeatures = df.select_dtypes(include=['float64', 'int64'])

# Korrelationstabelle berechnen
correlation_matrix = QuantitativeFeatures.corr(method='pearson')
print("Korrelationstabelle:\n", correlation_matrix)


In [ ]:
# Scatter-Matrix für alle quantitativen Features
QuantitativeFeatures = ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']

scatter_matrix(df[QuantitativeFeatures], figsize=(8, 8), diagonal='hist')
plt.show()


In [ ]:
# Zusammenfassung der Preisdaten pro `cut`
print(df.groupby('cut')['price'].describe())

# Zusammenfassung der Preisdaten pro `color`
print(df.groupby('color')['price'].describe())

# Zusammenfassung der Preisdaten pro `clarity`
print(df.groupby('clarity')['price'].describe())


In [ ]:
# Boxplot für Preis pro `cut`
df.boxplot(column=['price'], by='cut', figsize=(6, 4))
plt.title("Preisverteilung pro 'cut'")
plt.suptitle("")  # Entfernt den Standardtitel
plt.xlabel("Cut")
plt.ylabel("Price")
plt.show()

# Boxplot für Preis pro `color`
df.boxplot(column=['price'], by='color', figsize=(6, 4))
plt.title("Preisverteilung pro 'color'")
plt.suptitle("")
plt.xlabel("Color")
plt.ylabel("Price")
plt.show()

# Boxplot für Preis pro `clarity`
df.boxplot(column=['price'], by='clarity', figsize=(6, 4))
plt.title("Preisverteilung pro 'clarity'")
plt.suptitle("")
plt.xlabel("Clarity")
plt.ylabel("Price")
plt.show()


In [ ]:

# Boxplot für `carat` vs. `cut`
plt.figure(figsize=(6, 4))
df.boxplot(column=['carat'], by='cut')
plt.title("Karatverteilung pro 'cut'")
plt.suptitle("")
plt.xlabel("Cut")
plt.ylabel("Carat")
plt.show()

# Boxplot für `carat` vs. `color`
plt.figure(figsize=(6, 4))
df.boxplot(column=['carat'], by='color')
plt.title("Karatverteilung pro 'color'")
plt.suptitle("")
plt.xlabel("Color")
plt.ylabel("Carat")
plt.show()

# Boxplot für `carat` vs. `clarity`
plt.figure(figsize=(6, 4))
df.boxplot(column=['carat'], by='clarity')
plt.title("Karatverteilung pro 'clarity'")
plt.suptitle("")
plt.xlabel("Clarity")
plt.ylabel("Carat")
plt.show()


In [ ]:

# Barplot für `cut`
sns.countplot(data=df, x='cut')
plt.title("Häufigkeit der Kategorien in 'cut'")
plt.show()

# Boxplot für Preis pro `cut`
sns.boxplot(data=df, x='cut', y='price')
plt.title("Preisverteilung pro 'cut'")
plt.show()

### 5. Feature Engeneering

In [ ]:
print(df["cut"].unique())
print(df['clarity'].unique())
print(df['color'].unique())

In [ ]:
cut_categories = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
color_categories = ['J', 'I', 'H', 'G', 'F', 'E', 'D']
clarity_categories = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']

encoder = OrdinalEncoder(categories=[cut_categories, clarity_categories, color_categories])

ordinal_encoded = encoder.fit_transform(df[['cut', 'clarity', 'color']])


ordinal_encoded_df = pd.DataFrame(ordinal_encoded, columns = encoder.get_feature_names_out(['cut','clarity', 'color']))
print(ordinal_encoded_df.head())


df_ordinal_encoded = pd.concat([df.drop(['cut', 'clarity', 'color'], axis=1), ordinal_encoded_df], axis=1)
df = df_ordinal_encoded
df.head(12)


In [ ]:
df['price_per_carat'] = df['price'] / df['carat']
df['volume'] = df['x'] * df['y'] * df['z']
df['form_factor'] = df['x'] / df['y']
df['price_to_volume'] = df['price'] / df['volume']
df['mean_dimension'] = (df['x'] + df['y'] + df['z']) / 3
df['table_to_depth_ratio'] = df['table'] / df['depth']
df.head()


In [1]:

correlation_matrix = df.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.show()

corr_with_target = correlation_matrix['price'].abs()
threshold = 0.1
selected_features = corr_with_target[corr_with_target > threshold].index.tolist()

df_selected = df[selected_features]
df_selected.head()


NameError: name 'df' is not defined

In [ ]:
df_selected = df_selected.drop(columns=['Unnamed: 0'], errors='ignore') #unnamed wird entfernt
df= df_selected
df.head()

In [286]:

y= df['price']                                                                                              #daten wurden in features x und zielwert y sowie in Trainings- und Testsets aufgeteilt 
X= df.drop(columns=['price'])

X_train, X_test, y_test, y_train = train_test_split(X,y, test_size=0.2, random_state=42)


In [ ]:

# Überprüfe die Daten auf fehlende Werte
print("Anzahl fehlender Werte in den Features:")
print(df.isnull().sum())

# Entferne Zeilen mit fehlenden Werten
df = df.dropna()

# Überprüfe die Daten auf nicht-numerische Werte
print("Datentypen der Features:")
print(df.dtypes)

# Initialisiere den StandardScaler
scaler = StandardScaler()




In [ ]:

# Ersetze unendliche Werte durch NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Überprüfe auf unendliche Werte
print("Anzahl unendlicher Werte in den Features:")
print(df.isin([np.inf, -np.inf]).sum())

# Entferne Zeilen mit NaN-Werten
df.dropna(inplace=True)

# Überprüfe die Daten auf nicht-numerische Werte
print("Datentypen der Features:")
print(df.dtypes)

# Initialisiere den StandardScaler
scaler = StandardScaler()

# Auswahl der Features und Zielwert
X = df.drop('price', axis=1)
y = df['price']

# Fitte und transformiere die Daten
X_scaled = scaler.fit_transform(X)

# Neuer DataFrame mit den skalierten Daten
df_scaled = pd.DataFrame(X_scaled, columns=X.columns)
df_scaled['price'] = y

# Zeige die ersten Zeilen des neuen DataFrames
print("Erste Zeilen des skalierten DataFrames:")
print(df_scaled.head())


In [ ]:

# Ersetze unendliche Werte durch NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Überprüfe auf unendliche Werte
print("Anzahl unendlicher Werte in den Features:")
print(df.isin([np.inf, -np.inf]).sum())

# Entferne Zeilen mit NaN-Werten
df.dropna(inplace=True)

# Überprüfe auf extreme Werte
print("Maximale Werte in den Features:")
print(df.max())

print("Minimale Werte in den Features:")
print(df.min())

# Korrigiere extreme Werte
extreme_cols = ['price_to_volume', 'table_to_depth_ratio']
for col in extreme_cols:
    median = df[col].median()
    df[col] = np.where(np.abs(df[col]) > 1e10, median, df[col])

# Initialisiere den StandardScaler
scaler = StandardScaler()

# Auswahl der Features und Zielwert
X = df.drop('price', axis=1)
y = df['price']

# Fitte und transformiere die Daten
X_scaled = scaler.fit_transform(X)

# Neuer DataFrame mit den skalierten Daten
df_scaled = pd.DataFrame(X_scaled, columns=X.columns)
df_scaled['price'] = y

# Zeige die ersten Zeilen des neuen DataFrames
print("Erste Zeilen des skalierten DataFrames:")
print(df_scaled.head())


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Ersetze unendliche Werte durch NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Überprüfe auf unendliche Werte
print("Anzahl unendlicher Werte in den Features:")
print(df.isin([np.inf, -np.inf]).sum())

# Entferne Zeilen mit NaN-Werten
df.dropna(inplace=True)

# Überprüfe auf extreme Werte
print("Maximale Werte in den Features:")
print(df.max())

print("Minimale Werte in den Features:")
print(df.min())

# Korrigiere extreme Werte
extreme_cols = ['price_to_volume', 'table_to_depth_ratio']
for col in extreme_cols:
    max_value = df[col].max()
    min_value = df[col].min()
    print(f"Max value for {col}: {max_value}")
    print(f"Min value for {col}: {min_value}")
    df[col] = np.where(df[col] > 1e4, 1e4, df[col])  # Clipping extreme high values
    df[col] = np.where(df[col] < -1e4, -1e4, df[col])  # Clipping extreme low values

# Initialisiere den StandardScaler
scaler = StandardScaler()

# Auswahl der Features und Zielwert
X = df.drop('price', axis=1)
y = df['price']

# Fitte und transformiere die Daten
X_scaled = scaler.fit_transform(X)

# Neuer DataFrame mit den skalierten Daten
df_scaled = pd.DataFrame(X_scaled, columns=X.columns)
df_scaled['price'] = y

# Zeige die ersten Zeilen des neuen DataFrames
print("Erste Zeilen des skalierten DataFrames:")
print(df_scaled.head())


In [ ]:
# Daten skalieren mit Standardization

scaler = StandardScaler()

X = df.drop('price', axis=1)
y = df['price']

X_scaled = scaler.fit_transform(X)

df_scaled = pd.DataFrame(X_scaled, columns=X.columns)
df_scaled['price'] = y

print("Erste Zeilen des skalierten DataFrames:")
df_scaled.head()

In [ ]:
# Daten Skalieren mit Minmax scaling 

scaler = MinMaxScaler()

X = df.drop('price', axis=1)
y = df['price']

X_scaled = scaler.fit_transform(X)

df_scaled = pd.DataFrame(X_scaled, columns=X.columns)
df_scaled['price'] = y

print("Erste Zeilen des skalierten DataFrames:")
df_scaled.head()
